In [8]:
spark.conf.set("mapreduce.input.fileinputformat.input.dir.recursive", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
SILVER_PATH = "s3://ibmrawaml/Spark_curated_day/"



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql import SparkSession, functions as F, Window

spark = SparkSession.builder.appName("aml_full_transform").getOrCreate()

# Confirm Glue Catalog is the metastore
spark.sql("SHOW DATABASES").show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|  namespace|
+-----------+
|aml_curated|
|    aml_dev|
|    aml_raw|
|    default|
+-----------+

## Reading every partition

In [9]:
df_raw = spark.table("aml_raw.catchall")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_raw.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- col1: string (nullable = true)
 |-- col2: string (nullable = true)
 |-- col3: string (nullable = true)
 |-- col4: string (nullable = true)
 |-- col5: string (nullable = true)
 |-- col6: double (nullable = true)
 |-- col7: string (nullable = true)
 |-- col8: double (nullable = true)
 |-- col9: string (nullable = true)
 |-- col10: string (nullable = true)
 |-- col11: integer (nullable = true)

In [11]:
df_parsed = (
    df_raw
      .withColumn(
          "parsed_ts",
          F.coalesce(                                # accept either format
              F.to_timestamp("col1", "yyyy/MM/dd HH:mm"),           # 2022/10/10 06:50
              F.to_timestamp("col1", "yyyy/MM/dd HH:mm:ss"),        # 2022/10/10 06:50:00
              F.to_timestamp("col1", "yyyy-MM-dd HH:mm:ss"))        # 2022-10-10 06:50:00
      )
      .filter(F.col("parsed_ts").isNotNull())         # drop malformed rows
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_part = (
    df_parsed
      .withColumn("yr",  F.year("parsed_ts").cast("int"))
      .withColumn("mo",  F.month("parsed_ts").cast("int"))
      .withColumn("day", F.dayofmonth("parsed_ts").cast("int"))
      .drop("col1")                                   # raw string no longer needed
      .withColumnRenamed("parsed_ts", "timestamp")    # final name
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_final = (
    df_part
      .withColumnRenamed("col2",  "from_bank")
      .withColumnRenamed("col3",  "from_account")
      .withColumnRenamed("col4",  "to_bank")
      .withColumnRenamed("col5",  "to_account")
      .withColumnRenamed("col6",  "amount_received")
      .withColumnRenamed("col7",  "receiving_currency")
      .withColumnRenamed("col8",  "amount_paid")
      .withColumnRenamed("col9",  "payment_currency")
      .withColumnRenamed("col10", "payment_format")
      .withColumnRenamed("col11", "is_laundering")
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_final.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+-------------------+----+---+---+
|from_bank|from_account|to_bank|to_account|amount_received|receiving_currency|amount_paid|payment_currency|payment_format|is_laundering|          timestamp|  yr| mo|day|
+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+-------------------+----+---+---+
|    00952|   8139F54E0|0111632| 8062C56E0|        5331.44|         US Dollar|    5331.44|       US Dollar|           ACH|            1|2022-08-09 05:14:00|2022|  8|  9|
|  0111632|   8062C56E0| 008456| 81363F620|        5602.59|         US Dollar|    5602.59|       US Dollar|           ACH|            1|2022-08-13 13:09:00|2022|  8| 13|
|  0118693|   823D5EB90| 013729| 801CF2E60|        1400.54|         US Dollar|    1400.54|       US Dollar|           ACH|            1|2022-08-15 07:

In [15]:
df_final.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

431109182

In [18]:
(df_final
   .repartition("yr", "mo", "day")              # shuffle only on keys
   .write
   .mode("overwrite")                           # replaces only touched y/m/d
   .option("compression", "snappy")
   .partitionBy("yr", "mo", "day")
   .parquet("s3://ibmrawaml/Spark_curated_day/"))   # <- NEW prefix

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
spark.read.parquet(SILVER_PATH).count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

431109182